# Logistic Regression TF

### Importing data

In [1]:
import pandas as pd
df = pd.read_excel (r'C:\Users\HP\Downloads\Dataset.xlsx')
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,INTENT,ID,Unnamed: 2
0,remove the formatting from this sentence,18,NaN
1,superscript the last two words,8,NaN
2,Please remove formatting from this page,18,NaN
3,Can you please start inserting bullets for sel...,19,NaN
4,left align the image,16,NaN


In [2]:
df = pd.DataFrame(df) 
df.drop(['Unnamed: 2'], axis=1)
df.dropna(subset=['INTENT'], inplace=True)
df = df[['ID', 'INTENT']]
df.head()

,ID,INTENT
0,18,remove the formatting from this sentence
1,8,superscript the last two words
2,18,Please remove formatting from this page
3,19,Can you please start inserting bullets for sel...
4,16,left align the image


### Preprocessing

In [3]:
import re

def tokenize(text):
    tokens=re.split('\W+',text)
    return tokens
df['intent_tokenized']=df['INTENT'].apply(lambda x:tokenize(x.lower()))
df.head()

,ID,INTENT,intent_tokenized
0,18,remove the formatting from this sentence,"[remove, the, formatting, from, this, sentence]"
1,8,superscript the last two words,"[superscript, the, last, two, words]"
2,18,Please remove formatting from this page,"[please, remove, formatting, from, this, page]"
3,19,Can you please start inserting bullets for sel...,"[can, you, please, start, inserting, bullets, ..."
4,16,left align the image,"[left, align, the, image, ]"


In [4]:
import nltk
#stopword=nltk.download('stopwords')
stopword=nltk.corpus.stopwords.words('english')
stopword=stopword+['can', 'change', 'could','four','kindly','let','paragraph', 'please','please', 'present', 'put','take','what','you','capital','change', 'changes', 'character', 'characters','color','content','dictation','first', 'five','full', 'image','last',  'letter', 'letters', 'line', 'lines','make','move', 'near', 'needed','open', 'page', 'paragraph','picture', 'place', 'position', 'present', 'previous', 'red','required','selected', 'sentence', 'sentences','symbol', 'table','take', 'text', 'textoutline', 'three', 'two','word', 'words']

def remove_stopword(tokenised_list):
    text=[word.lower() for word in tokenised_list if word.lower() not in stopword]
    return text
df['intent_nostop']=df['intent_tokenized'].apply(lambda x:remove_stopword(x))
df.head()


,ID,INTENT,intent_tokenized,intent_nostop
0,18,remove the formatting from this sentence,"[remove, the, formatting, from, this, sentence]","[remove, formatting]"
1,8,superscript the last two words,"[superscript, the, last, two, words]",[superscript]
2,18,Please remove formatting from this page,"[please, remove, formatting, from, this, page]","[remove, formatting]"
3,19,Can you please start inserting bullets for sel...,"[can, you, please, start, inserting, bullets, ...","[start, inserting, bullets]"
4,16,left align the image,"[left, align, the, image, ]","[left, align, ]"


In [5]:
ps=nltk.PorterStemmer()
def stemming(tokenised_list):
    text=[ps.stem(word) for word in tokenised_list ]
    return text
df['intent_nostop_stem']=df['intent_nostop'].apply(lambda x:stemming(x))
df.head()

,ID,INTENT,intent_tokenized,intent_nostop,intent_nostop_stem
0,18,remove the formatting from this sentence,"[remove, the, formatting, from, this, sentence]","[remove, formatting]","[remov, format]"
1,8,superscript the last two words,"[superscript, the, last, two, words]",[superscript],[superscript]
2,18,Please remove formatting from this page,"[please, remove, formatting, from, this, page]","[remove, formatting]","[remov, format]"
3,19,Can you please start inserting bullets for sel...,"[can, you, please, start, inserting, bullets, ...","[start, inserting, bullets]","[start, insert, bullet]"
4,16,left align the image,"[left, align, the, image, ]","[left, align, ]","[left, align, ]"


In [6]:
#nltk.download('wordnet')
lm=nltk.WordNetLemmatizer()
def lemmatizing(tokenised_list):
    text=[lm.lemmatize(word) for word in tokenised_list ]
    return text
df['intent_nostop_lem']=df['intent_nostop'].apply(lambda x:lemmatizing(x))
df.head()

,ID,INTENT,intent_tokenized,intent_nostop,intent_nostop_stem,intent_nostop_lem
0,18,remove the formatting from this sentence,"[remove, the, formatting, from, this, sentence]","[remove, formatting]","[remov, format]","[remove, formatting]"
1,8,superscript the last two words,"[superscript, the, last, two, words]",[superscript],[superscript],[superscript]
2,18,Please remove formatting from this page,"[please, remove, formatting, from, this, page]","[remove, formatting]","[remov, format]","[remove, formatting]"
3,19,Can you please start inserting bullets for sel...,"[can, you, please, start, inserting, bullets, ...","[start, inserting, bullets]","[start, insert, bullet]","[start, inserting, bullet]"
4,16,left align the image,"[left, align, the, image, ]","[left, align, ]","[left, align, ]","[left, align, ]"


In [7]:
stopwords_intent=[]
def clean_text_stem(text):
    tokens=re.split('\W+',text)
    [stopwords_intent.append(word) for word in tokens if word in stopword]
    text=[word.lower() for word in tokens if word.lower() not in stopword]
    return text

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect_stem=CountVectorizer(analyzer=clean_text_stem)
X_counts_stem=count_vect_stem.fit_transform(df['INTENT'])
print(X_counts_stem.shape)
l=count_vect_stem.get_feature_names()

(475, 49)


In [9]:
X_counts_stem_df=pd.DataFrame(X_counts_stem.toarray())
X_counts_stem_df.columns=count_vect_stem.get_feature_names()
X_counts_stem_df.head()
print(X_counts_stem.shape)
print(count_vect_stem.get_feature_names())

(475, 49)
['', 'add', 'align', 'bold', 'break', 'bullet', 'bulleted', 'bullets', 'centre', 'clear', 'commands', 'comment', 'delete', 'deleted', 'display', 'emphasize', 'end', 'formatting', 'go', 'halt', 'help', 'icon', 'insert', 'inserting', 'italic', 'italicize', 'italics', 'left', 'list', 'middle', 'new', 'next', 'pause', 'remove', 'reveal', 'right', 'shift', 'show', 'start', 'stop', 'strike', 'strikethrough', 'subscript', 'superscript', 'unbold', 'underline', 'undo', 'unemphasize', 'unitalicize']


In [10]:
X_features=X_counts_stem_df
X_features.head()

,,add,align,bold,break,bullet,bulleted,bullets,centre,clear,...,stop,strike,strikethrough,subscript,superscript,unbold,underline,undo,unemphasize,unitalicize
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
import numpy as np
Y=df['ID']
Y=np.array(Y)


In [12]:
x_orig = X_features.iloc[:, 1:].values 
x_orig

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]], dtype=int64)

In [13]:
y_orig = df.iloc[:, 0:1].values 


In [14]:
print(x_orig.shape)
print(y_orig.shape)

(475, 48)
(475, 1)


In [15]:
from sklearn.preprocessing import OneHotEncoder 
oneHot = OneHotEncoder() 
x = x_orig
  
# Encoding y_orig 
oneHot.fit(y_orig) 
y = oneHot.transform(y_orig).toarray() 
  
alpha, epochs = 0.0035, 5000
m, n = x.shape 

y

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

### Model training

In [16]:
import tensorflow as tf
X = tf.placeholder(tf.float32, [None, n],name="x") 
  

Y = tf.placeholder(tf.float32, [None, 26],name="y") 
  

W = tf.Variable(tf.zeros([n, 26]),name="weights") 
  

b = tf.Variable(tf.zeros([26]),name="bias") 

Instructions for updating:
Colocations handled automatically by placer.


In [17]:
Y_hat = tf.nn.softmax(tf.add(tf.matmul(X, W), b),name="output") 


In [19]:
cost = tf.square(Y_hat - Y, name="cost")
cost=tf.reduce_sum(cost)

In [20]:
optimizer = tf.train.GradientDescentOptimizer( 
         learning_rate = alpha).minimize(cost) 

In [21]:
init = tf.global_variables_initializer() 
saver = tf.train.Saver()


In [22]:
with tf.Session() as sess: 
      
  #  # Initializing the Variables 
    sess.run(init) 
      
    # Lists for storing the changing Cost and Accuracy in every Epoch 
    cost_history, accuracy_history = [], [] 
      
    # Iterating through all the epochs 
    for epoch in range(epochs+1): 
        cost_per_epoch = 0
          
        # Running the Optimizer 
        sess.run(optimizer, feed_dict = {X : x, Y : y}) 
          
        # Calculating cost on current Epoch 
        c = sess.run(cost, feed_dict = {X : x, Y : y}) 
          
        # Calculating accuracy on current Epoch 
        correct_prediction = tf.equal(tf.argmax(Y_hat, 1), 
                                          tf.argmax(Y, 1)) 
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, 
                                                 tf.float32)) 
      #    
       # # Storing Cost and Accuracy to the history 
        cost_history.append(c) 
        accuracy_history.append(accuracy.eval({X : x, Y : y}) * 100) 
          
    #    # Displaying result on current Epoch 
        if epoch % 100 == 0 and epoch != 0: 
            print("Epoch " + str(epoch) + " Cost: "
                            + str(cost_history[-1])+ " Acc: "
                            + str(accuracy_history[-1])) 
      
    Weight = sess.run(W) # Optimized Weight 
    Bias = sess.run(b)   # Optimized Bias 
      
    ## Final Accuracy 
    correct_prediction = tf.equal(tf.argmax(Y_hat, 1), 
                                      tf.argmax(Y, 1)) 
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,  
                                             tf.float32)) 
    print("\nAccuracy:", accuracy_history[-1], "%") 
    saver.save(sess,"/tmp/model3.ckpt")
    print("Model saved")


Accuracy: 74.94736909866333 %
Model saved


In [23]:
Bias

array([-7.2480907e-05,  1.2736765e-03,  2.8890448e-03,  1.0044521e-03,
        1.8121274e-03,  1.5429182e-03, -3.4171643e-04, -6.1094720e-04,
       -6.1094615e-04, -3.4171669e-04, -8.8017504e-04, -1.4186305e-03,
        7.3520711e-04,  7.3521334e-04, -8.8017766e-04, -1.4186372e-03,
        1.9674613e-04,  1.2736815e-03, -1.1493993e-03, -1.6878552e-03,
       -1.6878593e-03, -7.2485862e-05, -7.2485964e-05,  7.3520909e-04,
       -1.4186475e-03,  4.6597555e-04], dtype=float32)

In [24]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
print_tensors_in_checkpoint_file(file_name='/tmp/model3.ckpt', tensor_name='', all_tensors=True)

tensor_name:  bias
[-7.2480907e-05  1.2736765e-03  2.8890448e-03  1.0044521e-03
  1.8121274e-03  1.5429182e-03 -3.4171643e-04 -6.1094720e-04
 -6.1094615e-04 -3.4171669e-04 -8.8017504e-04 -1.4186305e-03
  7.3520711e-04  7.3521334e-04 -8.8017766e-04 -1.4186372e-03
  1.9674613e-04  1.2736815e-03 -1.1493993e-03 -1.6878552e-03
 -1.6878593e-03 -7.2485862e-05 -7.2485964e-05  7.3520909e-04
 -1.4186475e-03  4.6597555e-04]
tensor_name:  weights
[[-5.1775140e-05 -5.1775140e-05 -5.1775140e-05 ... -5.1775140e-05
  -5.1775140e-05 -5.1775140e-05]
 [-4.4526623e-04 -4.4526623e-04 -4.4526623e-04 ... -4.4526623e-04
  -4.4526623e-04 -4.4526623e-04]
 [-2.8994077e-04  4.2869831e-03  2.6715978e-03 ... -2.8994077e-04
  -2.8994077e-04 -2.8994077e-04]
 ...
 [ 4.6597635e-03 -1.8639051e-04 -1.8639051e-04 ... -1.8639051e-04
  -1.8639051e-04 -1.8639051e-04]
 [-4.1420113e-05 -4.1420113e-05  1.0355030e-03 ... -4.1420113e-05
  -4.1420113e-05 -4.1420113e-05]
 [-4.1420113e-05 -4.1420113e-05 -4.1420113e-05 ... -4.1420113

### Converting to tflite

In [28]:
with tf.Session() as sess:

    saver.restore(sess, "/tmp/model3.ckpt")
    print("Model restored.")
    tf.train.write_graph(sess.graph_def,"/tmp/","savegraph.pbtxt")

INFO:tensorflow:Restoring parameters from /tmp/model3.ckpt
Model restored.


In [29]:
from tensorflow.python.tools import freeze_graph

In [40]:
save_path="/tmp/"
model_name="model3"
input_graph_path=save_path+"savegraph.pbtxt"
input_model_path=save_path+"model3.ckpt"

In [41]:
input_saver_def_path=""
input_binary=False
output_node_names="output_main"
restore_op_name="save/restore_all"
filename_tensor_name="save/Const:0"
output_frozen_graph_name=save_path+"frozen_model_"+model_name+".pb"
clear_devices=True

In [42]:
freeze_graph.freeze_graph(input_graph_path,input_saver_def_path,input_binary,input_model_path,output_node_names,restore_op_name,filename_tensor_name,output_frozen_graph_name,clear_devices,"")

Instructions for updating:
Use tf.gfile.GFile.
INFO:tensorflow:Restoring parameters from /tmp/model2.ckpt
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.


In [43]:
graph_def_file=output_frozen_graph_name

In [44]:
input_arrays=["x"]
output_arrays=["output_main"]

In [45]:
converter=tf.contrib.lite.TocoConverter.from_frozen_graph(graph_def_file,input_arrays,output_arrays)

Instructions for updating:
Use `lite.TFLiteConverter.from_frozen_graph` instead.


In [46]:
tflite_model=converter.convert()

In [47]:
open("LR.tflite","wb").write(tflite_model)

9912

### Checking accuracy

In [93]:
interpreter = tf.lite.Interpreter(model_path="LR.tflite")
interpreter.allocate_tensors()

In [94]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [95]:
input_shape = input_details[0]['shape']
input_shape

array([ 1, 85])

In [104]:
input_details

[{'name': 'x',
  'index': 6,
  'shape': array([ 1, 85]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0)}]

In [105]:
interpreter.set_tensor(input_details[0]['index'], l)

In [106]:
output_details

[{'name': 'output_main',
  'index': 3,
  'shape': array([1, 1]),
  'dtype': numpy.int64,
  'quantization': (0.0, 0)}]

In [107]:
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)



[[0]]


In [140]:
id_intent_map={1:"undo",2:"bold",3:"unbold",4:"italics",5:"unitalics",6:"underline",7:"un-underline",8:"superscript",9:"un-superscript",10:"subscript",11:"un-subscript",12:"strike",13:"unstrike",14:"center align",15:"comment",16:"left align",17:"right align",18:"un-format",19:"insert bullet",20:"next bullet",21:"end bullet",22:"pause dictation",23:"stop dictation",24:"commands",25:"help",26:"delete"}

In [171]:
input={'str':["can you reomve italics from it bro"]}
x_df = pd.DataFrame(input)
x_str=count_vect_stem.transform(x_df['str'])
x_str_stem_df=pd.DataFrame(x_str.toarray())
x_str_stem_df.columns=count_vect_stem.get_feature_names()
x_ = x_str_stem_df.iloc[:, 1:].values 
x_=np.asarray(x_, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], x_)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)
result=output_data[0][0]
id_intent_map[result+1]

[[4]]


'unitalics'

### printing weights and bias in java format so it can be copied directly

In [30]:
Weight
Weight.shape

(48, 26)

In [31]:
Bias

array([-7.2480907e-05,  1.2736765e-03,  2.8890448e-03,  1.0044521e-03,
        1.8121274e-03,  1.5429182e-03, -3.4171643e-04, -6.1094720e-04,
       -6.1094615e-04, -3.4171669e-04, -8.8017504e-04, -1.4186305e-03,
        7.3520711e-04,  7.3521334e-04, -8.8017766e-04, -1.4186372e-03,
        1.9674613e-04,  1.2736815e-03, -1.1493993e-03, -1.6878552e-03,
       -1.6878593e-03, -7.2485862e-05, -7.2485964e-05,  7.3520909e-04,
       -1.4186475e-03,  4.6597555e-04], dtype=float32)

In [32]:
w=Weight

In [33]:
b=Bias

In [34]:
w=w.transpose()
w.shape

(26, 48)

In [35]:
b.reshape(1,26)

array([[-7.2480907e-05,  1.2736765e-03,  2.8890448e-03,  1.0044521e-03,
         1.8121274e-03,  1.5429182e-03, -3.4171643e-04, -6.1094720e-04,
        -6.1094615e-04, -3.4171669e-04, -8.8017504e-04, -1.4186305e-03,
         7.3520711e-04,  7.3521334e-04, -8.8017766e-04, -1.4186372e-03,
         1.9674613e-04,  1.2736815e-03, -1.1493993e-03, -1.6878552e-03,
        -1.6878593e-03, -7.2485862e-05, -7.2485964e-05,  7.3520909e-04,
        -1.4186475e-03,  4.6597555e-04]], dtype=float32)

In [38]:
j=-1

In [39]:
for i in count_vect_stem.get_feature_names():
    print("vocab["+str(j)+"] = \""+i+"\";")
    j=j+1

vocab[-1] = "";
vocab[0] = "add";
vocab[1] = "align";
vocab[2] = "bold";
vocab[3] = "break";
vocab[4] = "bullet";
vocab[5] = "bulleted";
vocab[6] = "bullets";
vocab[7] = "centre";
vocab[8] = "clear";
vocab[9] = "commands";
vocab[10] = "comment";
vocab[11] = "delete";
vocab[12] = "deleted";
vocab[13] = "display";
vocab[14] = "emphasize";
vocab[15] = "end";
vocab[16] = "formatting";
vocab[17] = "go";
vocab[18] = "halt";
vocab[19] = "help";
vocab[20] = "icon";
vocab[21] = "insert";
vocab[22] = "inserting";
vocab[23] = "italic";
vocab[24] = "italicize";
vocab[25] = "italics";
vocab[26] = "left";
vocab[27] = "list";
vocab[28] = "middle";
vocab[29] = "new";
vocab[30] = "next";
vocab[31] = "pause";
vocab[32] = "remove";
vocab[33] = "reveal";
vocab[34] = "right";
vocab[35] = "shift";
vocab[36] = "show";
vocab[37] = "start";
vocab[38] = "stop";
vocab[39] = "strike";
vocab[40] = "strikethrough";
vocab[41] = "subscript";
vocab[42] = "superscript";
vocab[43] = "unbold";
vocab[44] = "underline";
vo

In [40]:
for i in range(0,26):   
    print("b["+str(i)+"] = "+str(b[i])+";")

b[0] = -7.248091e-05;
b[1] = 0.0012736765;
b[2] = 0.0028890448;
b[3] = 0.0010044521;
b[4] = 0.0018121274;
b[5] = 0.0015429182;
b[6] = -0.00034171643;
b[7] = -0.0006109472;
b[8] = -0.00061094615;
b[9] = -0.0003417167;
b[10] = -0.00088017504;
b[11] = -0.0014186305;
b[12] = 0.0007352071;
b[13] = 0.00073521334;
b[14] = -0.00088017766;
b[15] = -0.0014186372;
b[16] = 0.00019674613;
b[17] = 0.0012736815;
b[18] = -0.0011493993;
b[19] = -0.0016878552;
b[20] = -0.0016878593;
b[21] = -7.248586e-05;
b[22] = -7.2485964e-05;
b[23] = 0.0007352091;
b[24] = -0.0014186475;
b[25] = 0.00046597555;


In [41]:
b.shape

(26,)

In [42]:
w.shape

(26, 48)

In [43]:
for i in range(0,26):
    for j in range(0,48):
        print("w["+str(i)+"]["+str(j)+"] = "+str(w[i][j])+";")

w[0][0] = -5.177514e-05;
w[0][1] = -0.00044526623;
w[0][2] = -0.00028994077;
w[0][3] = -2.0710057e-05;
w[0][4] = -0.00019674555;
w[0][5] = -1.0355028e-05;
w[0][6] = -0.0001863905;
w[0][7] = -0.0001760355;
w[0][8] = -0.0005177514;
w[0][9] = -0.0002174556;
w[0][10] = -0.00015532544;
w[0][11] = -0.00019674553;
w[0][12] = -1.0355028e-05;
w[0][13] = -7.24852e-05;
w[0][14] = -0.0001449704;
w[0][15] = -0.00010355029;
w[0][16] = -0.0002174556;
w[0][17] = -4.1420113e-05;
w[0][18] = -3.1065087e-05;
w[0][19] = -0.00013461537;
w[0][20] = -3.1065087e-05;
w[0][21] = -0.00021745558;
w[0][22] = -4.1420113e-05;
w[0][23] = -0.00016568045;
w[0][24] = -0.00011390531;
w[0][25] = -0.00016568045;
w[0][26] = -0.00013461537;
w[0][27] = -2.0710057e-05;
w[0][28] = -5.1775143e-05;
w[0][29] = -2.0710057e-05;
w[0][30] = -0.00012426033;
w[0][31] = -0.00013461537;
w[0][32] = -0.00086982246;
w[0][33] = -1.0355028e-05;
w[0][34] = -0.00019674553;
w[0][35] = -1.0355028e-05;
w[0][36] = -0.00010355029;
w[0][37] = -9.319526